In [26]:
import numpy as np
from scipy import sparse, stats
import pandas as pd
import bottleneck
import gc
import warnings
import scanpy as sc
import anndata as ad
import time
import os
from scipy import stats, sparse
import concurrent.futures
import statistics
import random
import functools
import threading
from scipy.stats import hypergeom
import concurrent.futures
from scipy.sparse import csr_matrix
from scipy.stats import zscore
from functools import lru_cache


from pyprojroot import here
import pymn
import resource

In [27]:
here()
base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"

In [28]:
#common universe of markers
cluster_annotations = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep="\t")
all_markers = set(cluster_annotations['cluster.markers'].str.split(',').explode().tolist())
all_Zeng_markers = [x for x in all_markers if pd.notna(x)]
all_Zeng_markers = set(all_Zeng_markers)
len(all_Zeng_markers)


2725

In [29]:
cluster_annotations

,cl,cluster_id,cluster_id_label,supertype_id,supertype_label,supertype_id_label,subclass_id,subclass_label,subclass_id_label,class_label,...,multiome.size,cluster.markers,merfish.markers,F,M,sex.bias,Dark,Light,nt_type_label,nt_type_combo_label
0,128,1,0001 CLA-EPd-CTX Car3 Glut_1,1,CLA-EPd-CTX Car3 Glut_1,0001 CLA-EPd-CTX Car3 Glut_1,1,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,IT-ET Glut,...,0,"Gm45623,Nr4a2,Lgr5,Synpr","Col24a1,Smoc2,Slc17a7,Mctp2,Irs4,Synpr",0.50,0.50,NaN,0.02,0.98,Glut,Glut
1,129,2,0002 CLA-EPd-CTX Car3 Glut_1,1,CLA-EPd-CTX Car3 Glut_1,0001 CLA-EPd-CTX Car3 Glut_1,1,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,IT-ET Glut,...,0,"Car3,Slc17a7,4930407I19Rik","Osr1,Kcnip1,Irs4",0.52,0.48,NaN,0.02,0.98,Glut,Glut
2,130,3,0003 CLA-EPd-CTX Car3 Glut_1,1,CLA-EPd-CTX Car3 Glut_1,0001 CLA-EPd-CTX Car3 Glut_1,1,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,IT-ET Glut,...,0,"Car3,Satb2,Ccbe1","Smoc2,Col24a1,Cntnap3,Ccbe1",0.44,0.56,NaN,0.04,0.96,Glut,Glut
3,143,4,0004 CLA-EPd-CTX Car3 Glut_1,1,CLA-EPd-CTX Car3 Glut_1,0001 CLA-EPd-CTX Car3 Glut_1,1,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,IT-ET Glut,...,0,"Car3,Rxfp1,Tspan11","Col24a1,Igfbp6,Rspo1,Synpr,Rorb",0.46,0.54,NaN,0.04,0.96,Glut,Glut
4,131,5,0005 CLA-EPd-CTX Car3 Glut_2,2,CLA-EPd-CTX Car3 Glut_2,0002 CLA-EPd-CTX Car3 Glut_2,1,CLA-EPd-CTX Car3 Glut,001 CLA-EPd-CTX Car3 Glut,IT-ET Glut,...,0,"Gm45623,Egr2","Egr2,Cbln1,Ccn4",0.65,0.35,NaN,0.01,0.99,Glut,Glut
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5323,14935,5324,5324 LQ_1,1202,LQ_1,1202 LQ_1,339,LQ,339 LQ,LQ,...,0,"Myoc,Dlgap2","Gfap,Slc17a7,Rspo1,Pde1a",0.36,0.64,NaN,0.40,0.60,NaN,NaN
5324,14854,5325,5325 LQ_1,1202,LQ_1,1202 LQ_1,339,LQ,339 LQ,LQ,...,0,"Slc36a2,Cxcl5,C4b","Gfap,Sp8,Fxyd6",0.21,0.79,NaN,0.11,0.89,NaN,NaN
5325,14950,5326,5326 LQ_1,1202,LQ_1,1202 LQ_1,339,LQ,339 LQ,LQ,...,0,"Aqp4,Sp8,Kif5a","Aqp4,Sp8,Bcl11b,Grin2c",0.51,0.49,NaN,0.35,0.65,NaN,NaN
5326,34381,5327,5327 LQ_1,1202,LQ_1,1202 LQ_1,339,LQ,339 LQ,LQ,...,0,NaN,NaN,0.65,0.35,NaN,0.03,0.97,NaN,NaN


In [30]:
file_path = base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.all_markers.txt"

# Write the markers to the text file
with open(file_path, 'w') as file:
    for string in all_Zeng_markers:
        file.write(string + '\n')
file_path        

'/vault/lfrench/mouse_brain_cluster_replicability/data//whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.all_markers.txt'

In [31]:
#load Macosko marker lists from preprint supplement table
Macosko_supplement_table = pd.read_excel(base_data_folder + "/whole_mouse_brain/processed/macosko/paper_supplements/41586_2023_6818_MOESM10_ESM.xlsx", sheet_name = "Whole Brain Set Cover")
#just the most specific markers
Macosko_supplement_table = Macosko_supplement_table[Macosko_supplement_table["Exclude up to N Nearest Neighbors"] == 0]

Macosko_supplement_table = Macosko_supplement_table[["Cell Type Name", "Optimally-Sized Gene List"]]
Macosko_markers = Macosko_supplement_table['Optimally-Sized Gene List'].str.split('|', expand=True)
Macosko_markers = pd.concat([Macosko_supplement_table['Cell Type Name'], Macosko_markers], axis=1)
Macosko_markers = pd.melt(Macosko_markers, id_vars = 'Cell Type Name', value_name='gene_symbol', var_name='order')
Macosko_markers['gene_symbol'] = Macosko_markers['gene_symbol'].str.replace("=.*", "", regex=True)
Macosko_markers.dropna(subset=['gene_symbol'], inplace=True)
Macosko_markers = Macosko_markers.rename(columns = {"gene_symbol":"gene"})
Macosko_markers = Macosko_markers.rename(columns = {"Cell Type Name":"Macosko_cluster"})
all_Macosko_markers = set(Macosko_markers.gene)
len(all_Macosko_markers)

2706

In [32]:
Macosko_markers

,Macosko_cluster,order,gene
0,Ex_Gbx2_Adam18_2,0,Sp9
1,Inh_Pax7_Hpse_2,0,Pde11a
3,Ex_Pitx2_Zbtb7c_3,0,Kcnq5
4,Inh_Gata3_Gxylt2_5,0,Ltk
5,Inh_Lhx6_Npy_3,0,Syt6
...,...,...,...
36867,Ex_Ebf2_Prox1_Pou4f3,7,Trhr
37890,Inh_Lhx1_Otx2_1,7,Ankfn1
38645,Ex_Pou4f1_Plscr5_1,7,Dpf3
42880,Inh_Lhx1_Otx2_1,8,Dcdc2a


In [33]:
#write out Macosko and Zeng markers for later tests using the Metaneighbor results

In [34]:
Macosko_markers.to_csv(base_data_folder + "/whole_mouse_brain/processed/macosko/paper_supplements/41586_2023_6818_MOESM10_ESM.filtered.csv", index=False, columns=[col for col in Macosko_markers.columns if col not in ['index', 'order']])


In [35]:
base_data_folder + "/whole_mouse_brain/processed/macosko/paper_supplements/"

'/vault/lfrench/mouse_brain_cluster_replicability/data//whole_mouse_brain/processed/macosko/paper_supplements/'

In [36]:
len(all_Macosko_markers.union(all_Zeng_markers))

3853

In [7]:
# Write the markers to the text file
file_path = base_data_folder + "/whole_mouse_brain/processed/macosko/paper_supplements/41586_2023_6818_MOESM10_ESM.0_markers.txt"
with open(file_path, 'w') as file:
    for string in all_Macosko_markers:
        file.write(string + '\n')

In [8]:
base_data_folder + "/whole_mouse_brain/processed/macosko/paper_supplements/41586_2023_6818_MOESM10_ESM.0_markers.txt"

'/vault/lfrench/mouse_brain_cluster_replicability/data//whole_mouse_brain/processed/macosko/paper_supplements/41586_2023_6818_MOESM10_ESM.0_markers.txt'

In [38]:
#Use the smaller subsetted file
all_Zeng_cpm = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome_3820_markers_only.cpm.h5ad", backed=True) 

In [39]:
adata_macosko = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/macosko/from_google_drive/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.h5ad", backed = "r")
gene_map = adata_macosko.var
Macosko_universe = set(gene_map['gene_name'].to_list())
Zeng_universe = set(all_Zeng_cpm.var.gene_symbol.to_list())

In [40]:
all_Macosko_markers = all_Macosko_markers.intersection(Zeng_universe)
all_Zeng_markers = all_Zeng_markers.intersection(Macosko_universe)

In [41]:
len(all_Macosko_markers)

2673

In [42]:
len(all_Zeng_markers)

2214

In [43]:
combined_universe_of_markers = all_Zeng_markers.union(all_Macosko_markers)
intersection_of_markers = all_Zeng_markers.intersection(all_Macosko_markers)

In [44]:
Macosko_markers

,Macosko_cluster,order,gene
0,Ex_Gbx2_Adam18_2,0,Sp9
1,Inh_Pax7_Hpse_2,0,Pde11a
3,Ex_Pitx2_Zbtb7c_3,0,Kcnq5
4,Inh_Gata3_Gxylt2_5,0,Ltk
5,Inh_Lhx6_Npy_3,0,Syt6
...,...,...,...
36867,Ex_Ebf2_Prox1_Pou4f3,7,Trhr
37890,Inh_Lhx1_Otx2_1,7,Ankfn1
38645,Ex_Pou4f1_Plscr5_1,7,Dpf3
42880,Inh_Lhx1_Otx2_1,8,Dcdc2a


In [45]:
all_clusters_Zeng = cluster_annotations.cl.tolist()

In [46]:
#make dict for cluster markers and cells in cluster
cluster_marker_dict_Zeng = dict()
cluster_marker_dict_Zeng_union_filtered = dict()
for cluster in all_clusters_Zeng:
    #get markers 
    cluster_markers = cluster_annotations[cluster_annotations.cl == int(cluster)][['cluster.markers']].iloc[0,0]
    if pd.isna(cluster_markers):
        continue
    cluster_markers = cluster_markers.split(',')
    cluster_markers = list(set(cluster_markers).intersection(combined_universe_of_markers))
    cluster_marker_dict_Zeng_union_filtered[str(cluster)] = cluster_markers
    cluster_markers = list(set(cluster_markers).intersection(intersection_of_markers))
    if len(cluster_markers) == 0:
        continue
    cluster_marker_dict_Zeng[str(cluster)] = cluster_markers

In [47]:
cluster_marker_dict_Zeng

{'128': ['Synpr', 'Lgr5', 'Nr4a2'],
 '129': ['Slc17a7', 'Car3'],
 '130': ['Car3', 'Satb2', 'Ccbe1'],
 '143': ['Rxfp1', 'Car3', 'Tspan11'],
 '131': ['Egr2'],
 '116': ['Gria1', 'Plpp4', 'C1ql3', 'Tpbgl'],
 '120': ['Scube2', 'Gpr88', 'Npsr1'],
 '121': ['Satb2', 'Npsr1', 'Klf5'],
 '122': ['Npsr1', 'Cdhr1'],
 '125': ['Osr1', 'Lcp1', 'Vwc2l', 'Lepr'],
 '126': ['C1ql3', 'Lpl', 'Zfhx4', 'Nr4a2'],
 '127': ['Ctsc', 'Fst'],
 '117': ['Sfrp2'],
 '118': ['Rab38', 'Chst9'],
 '119': ['Osr1', 'Egr2', 'Onecut1'],
 '148': ['Egr2', 'Rorb', 'Adgrd1', 'Ccdc80', 'Npsr1'],
 '149': ['Osr1', 'Tpbgl', 'Npsr1'],
 '159': ['Zan', 'Col14a1'],
 '160': ['Adarb2', 'Pparg', 'Bace2'],
 '162': ['Col14a1', 'Egr2'],
 '123': ['Blnk', 'Medag', 'Ntn4', 'Igf1'],
 '97': ['Tmem40', 'Inhbb', 'Trdn'],
 '115': ['Satb2', 'Dcn'],
 '165': ['Sfrp1', 'Dcn'],
 '111': ['Il1rapl2', 'Met', 'Gxylt2', 'C1ql3', 'Bmpr1b'],
 '147': ['Fibcd1', 'Dcn'],
 '158': ['Drd1', 'Pamr1'],
 '114': ['Satb2', 'Crispld2', 'Chst9'],
 '145': ['Fibcd1', 'Kirrel3', 

In [48]:
data = []
for cluster, genes in cluster_marker_dict_Zeng.items():
    for gene in genes:
        data.append({'Zeng_cl': cluster, 'gene': gene})

In [49]:
df = pd.DataFrame(data)

In [51]:
df.to_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.all_markers_long.csv", index=False)

In [10]:
len(cluster_marker_dict_Zeng)

NameError: name 'cluster_marker_dict_Zeng' is not defined

In [21]:
all_clusters_Macosko = Macosko_markers.Macosko_cluster.tolist()
cluster_marker_dict_Macosko = dict()
cluster_marker_dict_Macosko_union_filtered = dict()
for cluster in all_clusters_Macosko:
    #get markers 
    cluster_markers = Macosko_markers[Macosko_markers.Macosko_cluster == cluster].gene.tolist()
    cluster_markers = list(set(cluster_markers).intersection(combined_universe_of_markers))
    cluster_marker_dict_Macosko_union_filtered[str(cluster)] = cluster_markers
    cluster_markers = list(set(cluster_markers).intersection(intersection_of_markers))
    if len(cluster_markers) == 0:
        continue
    cluster_marker_dict_Macosko[cluster] = cluster_markers

In [22]:
len(cluster_marker_dict_Macosko)

4583

In [66]:
cluster_marker_dict_long_Macosko = pd.DataFrame([(k, v) for k, vs in cluster_marker_dict_Macosko.items() for v in vs], columns=['cluster', 'gene_symbol'])

# Create a new DataFrame with 1s where the values match
cluster_marker_dict_table_Macosko = cluster_marker_dict_long_Macosko.pivot_table(index='cluster', columns='gene_symbol', aggfunc='size', fill_value=0)


In [67]:
cluster_marker_dict_table_Macosko.shape

(4583, 1578)

In [68]:
cluster_marker_dict_long_Macosko.shape

(14240, 2)

In [79]:
cluster_marker_dict_long_Zeng = pd.DataFrame([(k, v) for k, vs in cluster_marker_dict_Zeng.items() for v in vs], columns=['cluster', 'gene_symbol'])

# Create a new DataFrame with 1s where the values match
cluster_marker_dict_table_Zeng = cluster_marker_dict_long_Zeng.pivot_table(index='cluster', columns='gene_symbol', aggfunc='size', fill_value=0)



In [80]:
cluster_marker_dict_table_Zeng.shape

(5189, 1578)

In [81]:
cluster_marker_dict_table_Zeng.shape

(5189, 1578)

In [82]:
cluster_marker_dict_table_Zeng.max().max()

1

In [83]:
cluster_marker_dict_table_Macosko.shape

(4583, 1578)

In [84]:
cluster_marker_dict_table_Macosko.max().max()

1

In [85]:
intersections = cluster_marker_dict_table_Zeng @ cluster_marker_dict_table_Macosko.T

In [86]:
intersections.shape

(5189, 4583)

In [87]:
Macosko_max_mean = intersections.max().mean()
Macosko_max_mean

1.2633646083351517

In [88]:
Zeng_max_mean = intersections.max(axis=1).mean()
Zeng_max_mean

1.2272114087492774

In [89]:
greater_than_one_overlap = (intersections >1).sum().sum()
greater_than_one_overlap

1826

In [90]:
greater_than_two_overlap = (intersections >2).sum().sum()
greater_than_two_overlap

30

In [91]:
(intersections >0).sum().sum()

284431

In [92]:
(intersections == 0).sum().sum()

23496756

In [ ]:
#how many parameter sets for each cluster test - use a lookup table? cache?

In [ ]:
#need to iterate each cell and get a p-value. Skip entries with value of one and zero? 

In [ ]:
#universe of genes is the shared whole genome! so 1/1 might be significant!

In [ ]:
#fuzzy adjusted rand index or omega index could work on the whole matrix

In [ ]:
#need cluster sizes without intersection

In [102]:
#make dict for cluster markers and cells in cluster
length_dict_Zeng = dict()
for cluster in all_clusters_Zeng:
    #get markers 
    cluster_markers = cluster_annotations[cluster_annotations.cl == int(cluster)][['cluster.markers']].iloc[0,0]
    if pd.isna(cluster_markers):
        continue
    cluster_markers = cluster_markers.split(',')
    cluster_markers = list(set(cluster_markers).intersection(combined_universe_of_markers))
    length_dict_Zeng[str(cluster)] = len(cluster_markers)


In [103]:
sum(length_dict_Zeng.values())

15803

In [104]:
len(combined_universe_of_markers)

3309

In [105]:
length_dict_Macosko = dict()
for cluster in all_clusters_Macosko:
    #get markers 
    cluster_markers = Macosko_markers[Macosko_markers.Macosko_cluster == cluster].gene.tolist()
    cluster_markers = list(set(cluster_markers).intersection(combined_universe_of_markers))
    length_dict_Macosko[cluster] = len(cluster_markers)

In [ ]:
#iterate each cell in the matrix and put it in a p-value matrix

In [97]:
@lru_cache(maxsize=None)
def hypergeom_test(total_population, total_successes, sample_size, successes_in_sample):
    # Compute the hypergeometric test
    p_value = hypergeom.sf(successes_in_sample - 1, total_population, total_successes, sample_size)
    return p_value

In [98]:
intersections.shape

(5189, 4583)

In [99]:
hyper_p_values = pd.DataFrame(index=intersections.index, columns=intersections.columns).fillna(pd.NA)

In [ ]:
#hyper_p_values= hyper_p_values.iloc[1:500,1:200]

In [100]:
intersections_test = intersections #.iloc[1:500,1:200]

In [106]:
%%time
total_population = 21205 #universe of all genes
for index, row in intersections_test.iterrows():
    # index is the row index
    # row is a Series representing the row
    Zeng_marker_size = length_dict_Zeng[index]
#    if pd.isna(Zeng_marker_size):
#        continue
    for column, intersect_size in row.items():
        Mac_marker_size = length_dict_Macosko[column]
        #use the memoized version
        hyper_p_values.at[index, column] = hypergeom_test(total_population, Mac_marker_size, Zeng_marker_size, intersect_size)


CPU times: user 3min 1s, sys: 57.9 ms, total: 3min 1s
Wall time: 3min 1s


In [109]:
hyper_p_values.shape

(5189, 4583)

In [110]:
num_tests = intersections.shape[0] * intersections.shape[1]

In [111]:
(hyper_p_values * num_tests < 0.05).sum().sum()

30

In [112]:
(hyper_p_values * hyper_p_values.shape[0] < 0.05).sum().sum()

1826

In [113]:
(hyper_p_values * hyper_p_values.shape[1] < 0.05).sum().sum()

1826

In [114]:
hyper_p_values.reset_index()

cluster,cluster,Astro_Agt_Sntg1,Astro_Emid1_Cd38,Astro_Emid1_Gdf10,Astro_Ephb1_Fzd2,Astro_Ephb1_Gfap,Astro_Fabp7_Hs3st3a1_1,Astro_Fabp7_Hs3st3a1_2,Astro_Fabp7_Hs3st3a1_3,Astro_Fabp7_Hs3st3a1_4,...,Ser_Fev_Chodl,Ser_Fev_Oas1e,Ser_Fev_Pth2r,Ser_Fev_Samd9l,Ser_Fev_Siglec15,Ser_Fev_Wfdc12,Ser_Nkx6-1_Trh,Tanycyte_Rax_Ccdc170,Tanycyte_Rax_Fndc3c1_1,Tanycyte_Rax_Fndc3c1_2
0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,10,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,100,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1001,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5184,995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5185,996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5186,997,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5187,998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [115]:
hyper_p_values_melted = pd.melt(hyper_p_values.reset_index(), id_vars='cluster', var_name='Macosko_cluster', value_name='p_value')


In [116]:
hyper_p_values_melted[hyper_p_values_melted.p_value * num_tests < 0.05]

,cluster,Macosko_cluster,p_value
795545,24472,Ex_C1ql2_Glp1r,0.0
1467579,4862,Ex_Dach2_Arhgap36_3,0.0
5167249,4780,Ex_Lhx9_Adgrg2_1,0.0
5201262,2614,Ex_Lhx9_C1ql4,0.0
6716171,2440,Ex_Nkx2-4_Calcr_2,0.0
7724474,3889,Ex_Phox2b_Adam33_3,0.0
8706430,5042,Ex_Plekhg1_Col27a1_Egflam_1,0.0
9110764,4667,Ex_Pou4f1_Plscr5_6,0.0
9850017,2143,Ex_Sim1_Bsx_1,0.0
10421516,2724,Ex_Skor1_Samd3_4,0.0


In [117]:
length_dict_Macosko["Inh_Tfap2b_Tll2_3"]

6

In [118]:
length_dict_Zeng["3552"]

5

In [119]:
hyper_p_values_melted_filtered = hyper_p_values_melted[hyper_p_values_melted.p_value * hyper_p_values.shape[1] < 0.1].copy()

In [120]:
hyper_p_values_melted_filtered = hyper_p_values_melted_filtered.rename(columns={'cluster':'Zeng_cluster' })

In [121]:
for index, row in hyper_p_values_melted_filtered.iterrows():
    hyper_p_values_melted_filtered.at[index, 'Macosko_marker_size'] = length_dict_Macosko[row['Macosko_cluster']]
    hyper_p_values_melted_filtered.at[index, 'Zeng_marker_size'] = length_dict_Zeng[row['Zeng_cluster']]
    hyper_p_values_melted_filtered.at[index, 'intersection'] = intersections.loc[row['Zeng_cluster'], row['Macosko_cluster']]
    
    
    

In [123]:
hyper_p_values_melted_filtered

,Zeng_cluster,Macosko_cluster,p_value,Macosko_marker_size,Zeng_marker_size,intersection
82291,5023,CholEx_Hmx2_A2m,0.0,4.0,4.0,2.0
101192,3286,CholEx_Hmx3_Grpr,0.0,3.0,3.0,2.0
137077,2879,CholEx_Irx2_Slc43a3_4,0.0,3.0,4.0,2.0
137079,2880,CholEx_Irx2_Slc43a3_4,0.0,3.0,4.0,2.0
137082,2884,CholEx_Irx2_Slc43a3_4,0.0,3.0,5.0,2.0
...,...,...,...,...,...,...
23518599,2767,Ng_Lhx1_Trp73,0.0,2.0,3.0,2.0
23684741,2860,SerEx_Gata3_Fev_1,0.0,3.0,4.0,2.0
23731433,2851,Ser_Fev_Chodl,0.0,3.0,3.0,2.0
23732519,3870,Ser_Fev_Chodl,0.0,3.0,3.0,2.0


In [124]:
#add in the marker sets? for the 1800? check for reciprocal top hit overlap
hyper_p_values_melted_filtered[["Macosko_marker_size", "Zeng_marker_size", 'intersection']].drop_duplicates().shape

(61, 3)

In [125]:
hyper_p_values_melted_filtered.sort_values('intersection')

,Zeng_cluster,Macosko_cluster,p_value,Macosko_marker_size,Zeng_marker_size,intersection
82291,5023,CholEx_Hmx2_A2m,0.0,4.0,4.0,2.0
16459740,1213,Inh_Lhx6_Nr2e1_1,0.0,4.0,4.0,2.0
16459734,1208,Inh_Lhx6_Nr2e1_1,0.0,4.0,4.0,2.0
16459733,1207,Inh_Lhx6_Nr2e1_1,0.0,4.0,4.0,2.0
16456862,3229,Inh_Lhx6_Npy_4,0.0,4.0,5.0,2.0
...,...,...,...,...,...,...
12980591,3502,Inh_Emx2_Irx4,0.0,4.0,5.0,3.0
23223732,3552,Inh_Tfap2b_Tll2_3,0.0,6.0,5.0,3.0
795545,24472,Ex_C1ql2_Glp1r,0.0,5.0,5.0,3.0
18253506,4416,Inh_Pax2_Adamts12,0.0,5.0,4.0,3.0


In [126]:
hyper_p_values_melted_filtered[hyper_p_values_melted_filtered.p_value * num_tests < 0.05].sort_values('intersection')

,Zeng_cluster,Macosko_cluster,p_value,Macosko_marker_size,Zeng_marker_size,intersection
795545,24472,Ex_C1ql2_Glp1r,0.0,5.0,5.0,3.0
23016145,3528,Inh_Tfap2b_Cdh23_1,0.0,4.0,4.0,3.0
21669166,911,Inh_Sall3_Trhr_1,0.0,6.0,4.0,3.0
21230947,34141,Inh_Pax8_Tac2,0.0,3.0,3.0,3.0
21105394,24744,Inh_Pax8_Serpinf1,0.0,5.0,8.0,3.0
20716699,2923,Inh_Pax7_Slc38a11,0.0,6.0,4.0,3.0
18253506,4416,Inh_Pax2_Adamts12,0.0,5.0,4.0,3.0
17224859,3255,Inh_Nkx2-2_Gata2_5,0.0,3.0,3.0,3.0
16464922,1207,Inh_Lhx6_Nr2e1_2,0.0,3.0,4.0,3.0
14599200,3229,Inh_Gata3_Qrfprl_3,0.0,5.0,5.0,3.0


In [ ]:
#use another relaxed threshold - the number of comparisons in the same region

In [127]:
hyper_p_values_melted_filtered.to_csv(os.path.join(here(), "results", "marker_results", "Marker_overlap_filtered.csv"), index=False)

In [128]:
hyper_p_values_melted.to_csv(os.path.join(here(), "results", "marker_results", "Marker_overlap_full" + ".csv.gz"), compression="gzip", index=False)

In [54]:
#redo intersections using specific regions - these are the remapped regions
base_results_folder = os.path.join(here(), "results")
Zeng_regions = pd.read_csv(base_results_folder + "/region_profile_compare/Zeng_region_counts.csv")
Macosko_regions = pd.read_csv(base_results_folder + "/region_profile_compare/Macosko_region_counts.csv")

In [68]:
Macosko_regions

,region_remap,ClusterNm,cell_count_Macosko
0,ACA,Astro_Agt_Sntg1,6
1,ACA,Astro_Emid1_Cd38,0
2,ACA,Astro_Emid1_Gdf10,0
3,ACA,Astro_Ephb1_Fzd2,12215
4,ACA,Astro_Ephb1_Gfap,402
...,...,...,...
95565,VISP,Ser_Fev_Wfdc12,0
95566,VISP,Ser_Nkx6-1_Trh,0
95567,VISP,Tanycyte_Rax_Ccdc170,0
95568,VISP,Tanycyte_Rax_Fndc3c1_1,0


In [59]:
Zeng_regions

,region_remap,cl,cell_count_Zeng
0,ACA,1,0
1,ACA,2,0
2,ACA,3,0
3,ACA,4,0
4,ACA,5,0
...,...,...,...
101227,VISP,34374,0
101228,VISP,34376,0
101229,VISP,34380,0
101230,VISP,34381,0


In [85]:
z_mat = (
    Zeng_regions
    .pivot_table(index="region_remap",
                 columns="cl",
                 values="cell_count_Zeng",
                 fill_value=0)
)

m_mat = (
    Macosko_regions
    .pivot_table(index="region_remap",
                 columns="ClusterNm",
                 values="cell_count_Macosko",
                 fill_value=0)
)

In [88]:
common_regions = z_mat.index.intersection(m_mat.index)
z_mat = z_mat.loc[common_regions]
m_mat = m_mat.loc[common_regions]

In [89]:
z_bin = (z_mat > 0).astype(int)          # region × cl
m_bin = (m_mat > 0).astype(int)          # region × ClusterNm

# 4) Matrix multiply to get cl × ClusterNm co-occurrence matrix
#    Entry (cl_i, ClusterNm_j) = number of regions where both are present
co_mat = z_bin.T.dot(m_bin)              # shape: (n_cl, n_ClusterNm)

# co_mat is a DataFrame with:
#   index  = cl
#   columns = ClusterNm
# and values = number of regions where they co-occur

# 5) Count how many cl–ClusterNm pairs co-occur in at least one region
pairs_mask = co_mat >= 1
n_pairs_with_overlap = pairs_mask.values.sum()

In [93]:
5189 * 4583 #number of full matrix tests

23781187

In [90]:
print("Number of cl–ClusterNm pairs with ≥1 shared region:", n_pairs_with_overlap)

Number of cl–ClusterNm pairs with ≥1 shared region: 12161241


In [97]:
hyper_p_values_melted_filtered = pd.read_csv(os.path.join(here(), "results", "marker_results", "Marker_overlap_filtered.csv"))

In [ ]:
hyper_p_values_melted_filtered 

In [101]:
hyper_p_values_melted_filtered.p_value * n_pairs_with_overlap

0       1.947151
1       0.486834
2       0.973637
3       0.973637
4       1.622677
          ...   
1821    0.162283
1822    0.973637
1823    0.486834
1824    0.486834
1825    0.486834
Name: p_value, Length: 1826, dtype: float64

In [100]:
hyper_p_values_melted_filtered[hyper_p_values_melted_filtered.p_value * n_pairs_with_overlap < 0.05].shape

(30, 5)

In [102]:
#still only 30 survive correction, makes sense as it's half the comparisons. 

In [ ]:
#try using the comparisons where the two clusters have the same most populated region

In [103]:
# For each cl: mark region(s) where cell_count_Zeng is maximal (>0)
z_col_max = z_mat.max(axis=0)                           # max per cl
z_bin_max = ((z_mat.eq(z_col_max)) & (z_mat > 0)).astype(int)

# For each ClusterNm: mark region(s) where cell_count_Macosko is maximal (>0)
m_col_max = m_mat.max(axis=0)                           # max per ClusterNm
m_bin_max = ((m_mat.eq(m_col_max)) & (m_mat > 0)).astype(int)

# 3) Align on the same set of regions
common_regions = z_bin_max.index.intersection(m_bin_max.index)
z_bin_max = z_bin_max.loc[common_regions]
m_bin_max = m_bin_max.loc[common_regions]

# 4) Matrix multiply to get cl × ClusterNm "shared max-region" matrix
#    Entry (cl_i, ClusterNm_j) = number of regions that are simultaneously
#    a max region for cl_i and a max region for ClusterNm_j.
co_mat_max = z_bin_max.T.dot(m_bin_max)                 # shape: (n_cl, n_ClusterNm)

# 5) Count how many cl–ClusterNm pairs share max region
pairs_mask_max = co_mat_max >= 1
n_pairs_with_shared_max_region = pairs_mask_max.values.sum()

print("Number of cl–ClusterNm pairs sharing ≥1 max region:",
      n_pairs_with_shared_max_region)


Number of cl–ClusterNm pairs sharing ≥1 max region: 4752950


In [104]:
hyper_p_values_melted_filtered[hyper_p_values_melted_filtered.p_value * n_pairs_with_shared_max_region < 0.05].shape

(38, 5)